In [ ]:
import sys
sys.path.append('../')

import yaml
import numpy as np

import mesher
import material
import utils
import homogenize
import voronoi
import matplotlib.pyplot as plt
_Ext = utils.Extent
import matplotlib as mpl
plt.rcParams["figure.figsize"] = (15,10)
mpl.rcParams['figure.dpi'] = 300
rng = np.random.default_rng(seed=10)

import matplotlib.patches as patches

In [ ]:
with open('../settings.yaml', 'r') as file:
  config = yaml.safe_load(file)

In [ ]:
voro_cfg = config['VORONOI_DATA']
mat_cfg = config['MATERIAL']

In [ ]:
beta_extent = _Ext(voro_cfg['thick_min'], voro_cfg['thick_max'])
orient_ext = _Ext(voro_cfg['orient_min'], voro_cfg['orient_max'])
aniso_ext = _Ext(voro_cfg['aniso_min'], voro_cfg['aniso_max'])

macro_bbox = voronoi.VoronoiExtent(x=_Ext(0., 3.),
                                   y=_Ext(0., 3.))
macro_mesh = mesher.Mesher(nelx=3,
                           nely=3,
                           bounding_box=macro_bbox)


nx, ny = 120, 120
cx_start, cx_end = int(0.34*nx), int(0.67*nx)
cy_start, cy_end = int(0.34*ny), int(0.67*ny)

nx_img = cx_end - cx_start
ny_img = cy_end - cy_start

img_mesh = mesher.Mesher(nelx=nx, nely=ny, bounding_box=macro_bbox)

voro_local_extent = voronoi.VoronoiExtent(x=_Ext(-0.5, 0.5),
                                          y=_Ext(-0.5, 0.5))

In [ ]:
struct_mat = material.MaterialConstants(youngs_modulus=mat_cfg['youngs_modulus'],
                                        poissons_ratio=mat_cfg['poissons_ratio'])

In [ ]:
H = homogenize.Homogenization(lx=1.,
                              ly=1.,
                              nelx=nx_img,
                              nely=ny_img,
                              phiInDeg=90.,
                              matProp=struct_mat,
                              penal=3.)

# Random voronoi parameters

In [ ]:
num_samples = 12000
num_cells_x, num_cells_y = voro_cfg['cells_per_elem']
min_seperation = voro_cfg['min_seperation']
cells_per_mstr = num_cells_x*num_cells_y

cx = np.zeros((num_samples, macro_mesh.num_elems, cells_per_mstr))
cy = np.zeros((num_samples, macro_mesh.num_elems, cells_per_mstr))

ckey, bkey, akey, okey = rng.spawn(4)

for s in range(num_samples):
  for neigh in range(macro_mesh.num_elems):
    cx[s,neigh,:], cy[s,neigh,:] = mesher.generate_randomly_perturbed_grid_points(
                                                 num_cells_x,
                                                 num_cells_y,
                                                 min_seperation,
                                                 voro_local_extent,
                                                 ckey)


beta = utils.unnormalize(bkey.random((num_samples,)), beta_extent)
aniso = utils.unnormalize(akey.random((num_samples,)), aniso_ext)
orient = utils.unnormalize(okey.random((num_samples,)), orient_ext)

# Generate homogenized data

In [ ]:
mstr_imgs = np.zeros((num_samples, nx_img, ny_img))
constitutive_chol_data = np.zeros((num_samples, 9))
vol_frac = np.zeros((num_samples))

plot_interval = num_samples/20

for s in range(num_samples):
  cx_glob, cy_glob = voronoi.compute_voronoi_cell_coordns_to_global_frame(
                              cx[s,:,:], cy[s,:,:], macro_mesh)
  
  density = voronoi.compute_voronoi_density_field(img_mesh,
                                                  cx_glob.reshape(-1),
                                                  cy_glob.reshape(-1),
                                                  beta[s],
                                                  orient[s],
                                                  aniso[s]
                                              ).reshape((img_mesh.nelx, img_mesh.nely)).T
  density_center = density[cx_start:cx_end, cy_start:cy_end]
  C = H.homogenize(density_center)
  L = np.linalg.cholesky(C)
  vf = np.mean(density_center)

  mstr_imgs[s, :, :] = density_center
  constitutive_chol_data[s, 0] = L[0,0]
  constitutive_chol_data[s, 1] = L[1,1]
  constitutive_chol_data[s, 2] = L[2,2]
  constitutive_chol_data[s, 3] = L[1,0]
  constitutive_chol_data[s, 4] = L[2,0]
  constitutive_chol_data[s, 5] = L[2,1]
  vol_frac[s] = vf
  if s%plot_interval == 0:
    print(f'{s}/{num_samples}')
    print('aniso', aniso[s])
    print('theta', orient[s])
    print('beta', beta[s])
    print('vf', vf)
    print('C_00', C[0, 0])
    print('C_11', C[1, 1])
    fig, ax = plt.subplots(1, 3)
    ax[0].scatter(cx_glob, cy_glob)
    rect = patches.Rectangle((1, 1), 1, 1, linewidth=1, edgecolor='r', facecolor='none')
    ax[0].add_patch(rect)
    ax[0].set_aspect('equal')
    ax[1].imshow(density, cmap='coolwarm', origin='lower')
    ax[2].imshow(density_center, cmap='coolwarm', origin='lower')
    plt.show()

In [ ]:
def save_gen_data():
  np.save('../data/cx_aniso_raw', cx)
  np.save('../data/cy_aniso_raw', cy)
  np.save('../data/beta_aniso_raw', beta)
  np.save('../data/orient_aniso_raw', orient)
  np.save('../data/aniso_raw', aniso)
  np.save('../data/vf_aniso_raw', vol_frac)
  np.save('../data/const_chol_aniso_raw', constitutive_chol_data)
  np.save('../data/mstr_imgs_aniso_raw', mstr_imgs)

save_gen_data()

# Retrieve C matrix from L matrix and plot 

In [ ]:

L = np.zeros((num_samples, 3, 3))
L[:,0,0] = constitutive_chol_data[:, 0]
L[:,1,1] = constitutive_chol_data[:, 1]
L[:,2,2] = constitutive_chol_data[:, 2]
L[:,1,0] = constitutive_chol_data[:, 3]
L[:,2,0] = constitutive_chol_data[:, 4]
L[:,2,1] = constitutive_chol_data[:, 5]

C_data = np.einsum('dij,djk->dik',L, L.transpose(0,2,1))

img = plt.hexbin(C_data[:,0,0], C_data[:,1,1], C=vol_frac, cmap='coolwarm', gridsize=20)
cbar = plt.colorbar(img)
cbar.ax.tick_params(labelsize=38, labelfontfamily="Times New Roman")  

cbar.set_label('Volume Fraction', fontname='Times New Roman', fontsize=35)
plt.xlabel('$C_{00}$', fontname='Times New Roman', fontsize=35)
plt.ylabel('$C_{11}$', fontname='Times New Roman', fontsize=35)
plt.xticks(np.arange(0, 1.1, 0.2), fontname='Times New Roman', fontsize=35)
plt.yticks(np.arange(0, 1.1, 0.2), fontname='Times New Roman', fontsize=35)

plt.gca().tick_params(axis='both', which='major', length=18, width=1)
plt.gca().tick_params(axis='both', which='minor', length=7, width=1)
plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(0.05)) 
plt.gca().xaxis.set_minor_locator(plt.MultipleLocator(0.05)) 

plt.show()

In [ ]:

plt.figure()
plt.scatter(orient, C_data[:, 0, 0])
plt.xlabel('theta')
plt.ylabel('C_00')
plt.show()

# Plot all the input and output features of the NN

In [ ]:
# Input features
fig, axs = plt.subplots(5,1)
for ftr,ttl,ax in zip(
  [cx, cy, beta, orient, aniso],
   ['cx','cy','beta', 'orient', 'aniso'],
   axs.reshape(-1)):
  ax.hist(ftr.reshape(-1))
  ax.set_title(ttl)
# plt.axis('equal')
plt.show()

# C components
fig, axs = plt.subplots(2,3)
for ftr,ttl,ax in zip(
  [C_data[:,0,0], C_data[:,1,1], C_data[:,2,2],
                 C_data[:,0,1], C_data[:,0,2], C_data[:,1,2]],
   ['C00','C11','C22','C01','C02','C12'],
   axs.reshape(-1)):
  ax.hist(ftr.reshape(-1))
  ax.set_title(ttl)
plt.show()

# output features
fig, axs = plt.subplots(2,3)
for ftr,ttl,ax in zip(
  range(7), ['L00','L11','L22','L10','L20','L21'],
   axs.reshape(-1)):
  ax.hist(constitutive_chol_data[:,ftr].reshape(-1))
  ax.set_title(ttl)
plt.show()

# Plot vol frac distribution

In [ ]:

plt.hist(vol_frac, bins=30, color='skyblue', edgecolor='black')

# Customize fonts to Times New Roman
plt.xlabel('Volume fraction', fontname='Times New Roman', fontsize=35)
plt.ylabel('Frequency', fontname='Times New Roman', fontsize=35)


# Adjust axis markers and labels
plt.xticks(np.arange(0, 1.1, 0.2), fontname='Times New Roman', fontsize=35)
plt.yticks(np.arange(0, 700, 100), fontname='Times New Roman', fontsize=35)
plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(20)) 
plt.gca().xaxis.set_minor_locator(plt.MultipleLocator(0.05)) 
plt.gca().tick_params(axis='both', which='major', length=18, width=1)
plt.gca().tick_params(axis='both', which='minor', length=7, width=1)
plt.show()
